## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database1.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,16.00,85,3,5.53,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,48.20,71,20,23.02,few clouds
2,2,Cape Town,ZA,-33.9258,18.4232,70.00,82,75,8.05,broken clouds
3,3,Georgetown,MY,5.4112,100.3354,82.99,57,20,9.22,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,86.00,79,75,13.80,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Georgetown,MY,5.4112,100.3354,82.99,57,20,9.22,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,86.00,79,75,13.80,broken clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,78.01,89,42,5.01,scattered clouds
15,15,Namatanai,PG,-3.6667,152.4333,84.58,66,100,13.51,overcast clouds
17,17,Rikitea,PF,-23.1203,-134.9692,79.50,70,0,11.32,clear sky
23,23,Avarua,CK,-21.2078,-159.7750,82.40,74,75,3.44,broken clouds
32,32,Alofi,NU,-19.0595,-169.9187,80.60,78,90,10.36,light rain
34,34,Atuona,PF,-9.8000,-139.0333,79.12,75,77,16.96,broken clouds
42,42,Grenville,GD,12.1167,-61.6167,78.80,83,20,17.27,few clouds
45,45,Muisne,EC,0.6000,-80.0333,77.18,88,19,3.47,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No need for this step since there are no null cells

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Georgetown,MY,82.99,few clouds,5.4112,100.3354,
4,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,
10,Puerto Ayora,EC,78.01,scattered clouds,-0.7393,-90.3518,
15,Namatanai,PG,84.58,overcast clouds,-3.6667,152.4333,
17,Rikitea,PF,79.50,clear sky,-23.1203,-134.9692,
23,Avarua,CK,82.40,broken clouds,-21.2078,-159.7750,
32,Alofi,NU,80.60,light rain,-19.0595,-169.9187,
34,Atuona,PF,79.12,broken clouds,-9.8000,-139.0333,
42,Grenville,GD,78.80,few clouds,12.1167,-61.6167,
45,Muisne,EC,77.18,light rain,0.6000,-80.0333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Georgetown,MY,82.99,few clouds,5.4112,100.3354,Cititel Penang
4,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Puerto Ayora,EC,78.01,scattered clouds,-0.7393,-90.3518,Castillo Galapagos
15,Namatanai,PG,84.58,overcast clouds,-3.6667,152.4333,Namatanai Hotel
17,Rikitea,PF,79.50,clear sky,-23.1203,-134.9692,Pension Maro'i
23,Avarua,CK,82.40,broken clouds,-21.2078,-159.7750,Paradise Inn
32,Alofi,NU,80.60,light rain,-19.0595,-169.9187,Taloa Heights
34,Atuona,PF,79.12,broken clouds,-9.8000,-139.0333,Villa Enata
42,Grenville,GD,78.80,few clouds,12.1167,-61.6167,Valley Breeze Guest House
45,Muisne,EC,77.18,light rain,0.6000,-80.0333,Quinta Samay


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Georgetown,MY,82.99,few clouds,5.4112,100.3354,Cititel Penang
4,Vaini,TO,86.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Puerto Ayora,EC,78.01,scattered clouds,-0.7393,-90.3518,Castillo Galapagos
15,Namatanai,PG,84.58,overcast clouds,-3.6667,152.4333,Namatanai Hotel
17,Rikitea,PF,79.50,clear sky,-23.1203,-134.9692,Pension Maro'i
...,...,...,...,...,...,...,...
673,Santa Rosa,PH,81.00,scattered clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home
677,Dingle,PH,80.69,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
678,Dagua,CO,77.00,clear sky,3.6569,-76.6886,Hostal Río Dagua
679,Makakilo City,US,82.40,scattered clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))